In [50]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [51]:
tf.logging.set_verbosity(tf.logging.INFO)

tf.app.flags.DEFINE_integer('max_steps', 10000, 'The number of steps to train a model')
tf.app.flags.DEFINE_string('model_dir', '/tmp/iris/models/ckpt/', 'Dir to save a model and checkpoints')
FLAGS = tf.app.flags.FLAGS

INPUT_FEATURE = 'x'
NUM_CLASSES = 3

In [52]:
def serving_input_receiver_fn():
    """
    This is used to define inputs to serve the model.
    :return: ServingInputReciever
    """
    receiver_tensors = {
        'sepal_length': tf.placeholder(tf.float32, [None, 1]),
        'sepal_width': tf.placeholder(tf.float32, [None, 1]),
        'petal_length': tf.placeholder(tf.float32, [None, 1]),
        'petal_width': tf.placeholder(tf.float32, [None, 1]),
    }

    # Convert give inputs to adjust to the model.
    features = {
        INPUT_FEATURE: tf.concat([
            receiver_tensors['sepal_length'],
            receiver_tensors['sepal_width'],
            receiver_tensors['petal_length'],
            receiver_tensors['petal_width']
        ], axis=1)
    }
    return tf.estimator.export.ServingInputReceiver(receiver_tensors=receiver_tensors,
                                                    features=features)

In [53]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Split data into train and eval.
index_list = range(len(y))
index_train, index_eval = train_test_split(index_list, train_size=0.8)
X_train, X_eval = X[index_train], X[index_eval]
y_train, y_eval = y[index_train], y[index_eval]

# Define the feature columns for inputs.
feature_columns = [
    tf.feature_column.numeric_column(INPUT_FEATURE, shape=[4])
]

/home/jugs/PycharmProjects/myvenv/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2178: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [54]:
feature_columns

[NumericColumn(key='x', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [55]:
training_config = tf.estimator.RunConfig(
        model_dir='/tmp/iris/models/ckpt/',
        save_summary_steps=100,
        save_checkpoints_steps=100)

In [56]:
classifier = tf.estimator.DNNClassifier(
        config=training_config,
        feature_columns=feature_columns,
        hidden_units=[10, 20, 10],
        n_classes=NUM_CLASSES,
        model_dir='/tmp/iris/models/ckpt/',
    )

INFO:tensorflow:Using config: {'_model_dir': '/tmp/iris/models/ckpt/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6bb23afe10>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [57]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={INPUT_FEATURE: X_train},
        y=y_train,
        # batch_size=5,
        num_epochs=1000,
        shuffle=True)

In [58]:
train_spec = tf.estimator.TrainSpec(
        input_fn=train_input_fn,
        max_steps=10000)

In [59]:
latest_exporter = tf.estimator.LatestExporter(
        name="models",
        serving_input_receiver_fn=serving_input_receiver_fn,
        exports_to_keep=10)

In [60]:
best_exporter = tf.estimator.BestExporter(
        serving_input_receiver_fn=serving_input_receiver_fn,
        exports_to_keep=1)

In [61]:
exporters = [latest_exporter, best_exporter]

In [62]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={INPUT_FEATURE: X_eval},
        y=y_eval,
        num_epochs=1,
        shuffle=False)

In [63]:
eval_spec = tf.estimator.EvalSpec(
        input_fn=eval_input_fn,
        throttle_secs=180,
        steps=10,
        exporters=exporters)

In [64]:
tf.estimator.train_and_evaluate(classifier, train_spec=train_spec, eval_spec=eval_spec)

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 100 or save_checkpoints_secs None.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Don

INFO:tensorflow:Skip the current checkpoint eval due to throttle secs (180 secs).
INFO:tensorflow:global_step/sec: 707.124
INFO:tensorflow:loss = 8.628676, step = 501 (0.142 sec)
INFO:tensorflow:Saving checkpoints for 600 into /tmp/iris/models/ckpt/model.ckpt.
INFO:tensorflow:Skip the current checkpoint eval due to throttle secs (180 secs).
INFO:tensorflow:global_step/sec: 922.569
INFO:tensorflow:loss = 15.100631, step = 601 (0.108 sec)
INFO:tensorflow:Saving checkpoints for 700 into /tmp/iris/models/ckpt/model.ckpt.
INFO:tensorflow:Skip the current checkpoint eval due to throttle secs (180 secs).
INFO:tensorflow:global_step/sec: 920.971
INFO:tensorflow:loss = 4.399117, step = 701 (0.109 sec)
INFO:tensorflow:Saving checkpoints for 800 into /tmp/iris/models/ckpt/model.ckpt.
INFO:tensorflow:Skip the current checkpoint eval due to throttle secs (180 secs).
INFO:tensorflow:global_step/sec: 952.541
INFO:tensorflow:loss = 2.0236015, step = 801 (0.105 sec)
INFO:tensorflow:Saving checkpoints f

({'accuracy': 1.0,
  'average_loss': 0.01569277,
  'loss': 0.4707831,
  'global_step': 938},
 [b'/tmp/iris/models/ckpt/export/models/1572772739',
  b'/tmp/iris/models/ckpt/export/best_exporter/1572772740'])

# Model is being saved to the export path directory
# We can check the signature using saved_model_cli 

In [66]:
!saved_model_cli show --dir /tmp/iris/models/ckpt/export/models/1572772739/ --all

/home/jugs/PycharmProjects/myvenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jugs/PycharmProjects/myvenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jugs/PycharmProjects/myvenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jugs/PycharmProjects/myvenv/lib/python3.6/s

# The model is ready for serving, we can serve the model using tensorflow_model_server api. Do the serving in the diffrent terminal window, to let it serve until we kill the session

In [17]:
!tensorflow_model_server  --port=8502 --rest_api_port=8503 --model_name=iris --model_base_path=/tmp/iris/models/ckpt/export/models/

# Now lets write the client section for the inferencing the serve model. 

In [1]:
import json
import requests
import os
import json

In [2]:
endpoint = "http://localhost:8501/v1/models/irisflower:predict"
headers = {"content-type":"application-json"}

In [3]:
instances = [{"sepal_length":[6.8],"sepal_width":[3.2],"petal_length":[5.9],"petal_width":[2.3]}]

In [4]:
data = json.dumps({"signature_name":"predict","instances": instances})

In [5]:
response = requests.post(endpoint, data=data, headers=headers)
prediction = json.loads(response.text)['predictions']
prediction

[{'class_ids': [2],
  'classes': ['2'],
  'all_class_ids': [0, 1, 2],
  'all_classes': ['0', '1', '2'],
  'probabilities': [3.69442699e-10, 0.000672064896, 0.999328],
  'logits': [-13.0567713, 1.35709763, 8.66158104]}]

In [13]:
instances = [{"examples":[{"age":50,"workclass":"Self-emp-not-inc","education":"Bachelors","education-num":13,"marital-status":"Married-civ-spouse","occupation":"Exec-managerial","relationship":"Husband", "race":"White", "sex":"Male", "capital-gain":0.0, "capital-loss":0.0, "hours-per-week":13, "native-country":"United-States"}]}]

In [14]:
instances = [{"examples": [{"age":50,"workclass":"Self-emp-not-inc","education":"Bachelors","education-num":13,"marital-status":"Married-civ-spouse","occupation":"Exec-managerial","relationship":"Husband", "race":"White", "sex":"Male", "capital-gain":0, "capital-loss":0, "hours-per-week":13, "native-country":"United-States"}]}]

In [20]:
endpoint = "http://localhost:8501/v1/models/census:predict"
headers = {"content-type":"application-json"}

In [24]:
data = json.dumps({"signature_name":"predict","instances": instances})

In [17]:
data

'{"signature_name": "serving-default", "instances": [{"examples": [{"age": 50, "workclass": "Self-emp-not-inc", "education": "Bachelors", "education-num": 13, "marital-status": "Married-civ-spouse", "occupation": "Exec-managerial", "relationship": "Husband", "race": "White", "sex": "Male", "capital-gain": 0, "capital-loss": 0, "hours-per-week": 13, "native-country": "United-States"}]}]}'

In [26]:
response = requests.post(endpoint, data=data, headers=headers)
prediction = json.loads(response.text)["predictions"]
prediction

JSONDecodeError: Invalid \escape: line 1 column 59 (char 58)

In [4]:
39, Local-gov, 170263, Masters, 14, Divorced, Prof-specialty, Unmarried, White, Female, 0, 0, 30, United-States
24, Private, 300275, HS-grad, 9, Never-married, Handlers-cleaners, Not-in-family, White, Male, 0, 0, 40, United-States

In [27]:
!saved_model_cli show --dir /home/jugs/PycharmProjects/DelvifyFWD/TFX_Module/census_data/exported_model_dir/1572768953 --all

/home/jugs/PycharmProjects/myvenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jugs/PycharmProjects/myvenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jugs/PycharmProjects/myvenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jugs/PycharmProjects/myvenv/lib/python3.6/s